# Artificial Neural Network (ANN) _from Scratch_

## Import required libraries

In [1]:
import numpy as np
from abc import ABC, abstractmethod

In [2]:
np.random.seed(37)

## Activation Functions

In [3]:
class Activation(ABC):

    @abstractmethod
    def __call__(self, z: np.ndarray) -> np.ndarray:
        pass

    @abstractmethod
    def derivative(self, z: np.ndarray) -> np.ndarray:
        pass

### ReLU

In [4]:
class ReLU(Activation):
    def __call__(self, z: np.ndarray) -> np.ndarray:
        return np.maximum(0, z)

    def derivative(self, z: np.ndarray) -> np.ndarray:
        return (z > 0).astype(float)

### Sigmoid

In [5]:
class Sigmoid(Activation):
    def __call__(self, z: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-z))

    def derivative(self, z: np.ndarray) -> np.ndarray:
        sigmoid = self(x)
        return sigmoid * (1 - sigmoid)

### Softmax

In [6]:
class Softmax(Activation):
    def __call__(self, z: np.ndarray) -> np.ndarray:
        exps = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def derivative(self, z: np.ndarray) -> np.ndarray:
        z = z.reshape(-1, 1)
        return np.diagflat(z) - np.dot(z, z.T)

### Tanh

In [7]:
class Tanh:
    def __call__(self, z: np.ndarray) -> np.ndarray:
        return np.tanh(z)

    def derivative(self, z: np.ndarray) -> np.ndarray:
        tanh = self(z)
        return 1 - tanh ** 2

## Loss Functions

In [9]:
class Loss(ABC):
    
    @abstractmethod
    def __call__(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        pass

    @abstractmethod
    def derivative(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        pass

### Mean Squared Error (MSE)

In [11]:
class MSE(Loss):
    def __call__(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        return np.mean((y_pred - y_true) ** 2) / 2

    def derivative(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        return (y_pred - y_true) / y_true.shape[0]

## Cross Entropy Loss (Categorical)

In [12]:
class CategoricalCrossEntropy(Loss):
    def __call__(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        y_pred = np.clip(y_pred, 1e-9, 1 - 1e-9)  # prevent log(0)
        return -np.sum(y_true * np.log(y_pred)) / y_true.shape[0]

    def derivative(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        return (y_pred - y_true) / y_true.shape[0]

## Layer

In [13]:
class Layer(ABC):
    
    @abstractmethod
    def __init__(self, in_dim: int, out_dim: int):
        pass

    @abstractmethod
    def forward(self, x: np.ndarray) -> np.ndarray:
        pass

    @abstractmethod
    def backward(self, dA: np.ndarray, lr: float) -> np.ndarray:
        pass

### Dense Layer

In [14]:
class Dense:
    def __init__(self, in_dim: int, out_dim: int, activation: Activation):
        self.weights = np.random.random(in_dim, out_dim)
        self.bias = np.zeros((1, out_dim))
        self.activation = activation

        self.x = None
        self.z = None

    def forward(self, x: np.ndarray) -> np.ndarray:
        self.x = x
        self.z = self.x @ self.weights + self.bias
        return self.activation(self.z)

    def backward(self, dA: np.ndarray, lr: np.ndarray) -> np.ndarray:
        dz = dA * self.activation.derivative(self.z)
        dw = self.x.T @ dz
        db = np.sum(dz, axis=0, keepdims=True)

        self.weights -= lr * dw
        self.bias -= lr * db
        return dz @ self.weights.T

    @staticmethod
    def params_(self):
        return np.array([self.weights, self.bias])